In [1]:
%matplotlib inline

from os import listdir, path
from os.path import isfile, join
from tqdm import tqdm
import json
from datetime import datetime

sexpr_folder = './sexprs/'
onlyfiles = [sexpr_folder + f for f in listdir(sexpr_folder) if isfile(join(sexpr_folder, f))]

In [2]:
print(f"Path count: {len(onlyfiles)}")
print(f"Total Size: {sum([path.getsize(x) for x in onlyfiles]) // 1024}kb")

Path count: 98
Total Size: 1205kb


In [3]:
group_by_program = {}
def get_file(filepath):
    filename = filepath.split('/')[2] #Get the file
    full_filename = filename.split('.')[0]
    filename = full_filename.split('_')
    with open(filepath, 'rb') as fileobj:
        content = fileobj.read()
        contents = json.loads(content)
        return {
            'name': full_filename,
            'program_hash': filename[0],
            'invocation': int(filename[1][1:]),
            'pc': int(filename[2][2:]),
            'solver_count': int(filename[3]),
            'solve_time': int(filename[4]),
            'sexpr': contents['sexprs'],
            'stats': contents['stats']
        }
all_sexprs = [get_file(filepath) for filepath in onlyfiles]

In [4]:
from z3 import Solver, SolverFor

In [5]:
logic_list = ["AUFLIA", "AUFLIRA", "AUFNIRA", "LRA", "QF_ABV", "QF_AUFBV", "QF_UFBV", "QF_AUFLIA", "QF_AX", "QF_BV", "QF_IDL", "QF_RDL", "QF_LIA", "QF_LRA", "QF_NIA", "QF_NRA", "QF_UF", "QF_UFIDL", "QF_UFLIA", "QF_UFLRA", "QF_UFNRA", "UFLRA", "UFNIA", "UFBV", "QF_S"]
logics_to_test = [('default', None)] + [(x, x) for x in logic_list]

In [8]:
hardest_problem = sorted(all_sexprs, key=lambda x: -x['solve_time'])[0]

In [9]:
hardest_problem

{'name': 'f953f3189c06bec607c016363021a2a5_i1_pc156_97_276182',
 'program_hash': 'f953f3189c06bec607c016363021a2a5',
 'invocation': 1,
 'pc': 156,
 'solver_count': 97,
 'solve_time': 276182,
 'sexpr': '(declare-fun CallValue_0 () (_ BitVec 256))\n(declare-fun AttackerWalletStarting () (_ BitVec 256))\n(declare-fun CallDataSize_0 () Int)\n(declare-fun input_0_0 () (_ BitVec 256))\n(declare-fun input@0_4 () (_ BitVec 256))\n(declare-fun input_0_32 () (_ BitVec 256))\n(declare-fun Timestamp_0 () (_ BitVec 256))\n(declare-fun Timestamp_1 () (_ BitVec 256))\n(declare-fun CallValue_1 () (_ BitVec 256))\n(declare-fun CallDataSize_1 () Int)\n(declare-fun input_1_0 () (_ BitVec 256))\n(declare-fun input@1_4 () (_ BitVec 256))\n(declare-fun input_1_32 () (_ BitVec 256))\n(declare-fun AttackerWallet () (_ BitVec 256))\n(declare-fun LastReturnType () Int)\n(declare-fun FirstTimestamp () (_ BitVec 256))\n(declare-fun LastTimestamp () (_ BitVec 256))\n(declare-fun |61747461636b6572000000000000000000

In [11]:
from z3 import parse_smt2_string

def test_sexpr_with_logic(file_contents, logic):
    if logic is None:
        solver = Solver()
    else:
        solver = SolverFor(logic)
    solver.add(parse_smt2_string(file_contents['sexpr']))
    start_time = datetime.now()
    result = solver.check()
    end_time = datetime.now()
    
    return {
        'sexpr_set': file_contents['name'],
        'logic': logic,
        'solver': solver,
        'check_time': (end_time - start_time).total_seconds(),
        'check_result': result
    }

test_results = []

for problem in tqdm(all_sexprs):
    result_set = []
    for logic_name, logic in tqdm(logics_to_test):
        result = test_sexpr_with_logic(problem, logic)
        result_set.append(result)
    test_results.append(result_set)



  0%|          | 0/98 [00:00<?, ?it/s]


  0%|          | 0/26 [00:00<?, ?it/s]


 73%|███████▎  | 19/26 [00:00<00:00, 189.70it/s]


100%|██████████| 26/26 [00:00<00:00, 150.06it/s]

  1%|          | 1/98 [00:00<00:17,  5.58it/s]


  0%|          | 0/26 [00:00<?, ?it/s]


 81%|████████  | 21/26 [00:00<00:00, 191.68it/s]


100%|██████████| 26/26 [00:00<00:00, 179.56it/s]

  2%|▏         | 2/98 [00:00<00:16,  5.87it/s]


  0%|          | 0/26 [00:00<?, ?it/s]


  4%|▍         | 1/26 [00:00<00:03,  6.96it/s]


 31%|███       | 8/26 [00:00<00:01,  9.52it/s]


 62%|██████▏   | 16/26 [00:00<00:00, 12.80it/s]


 85%|████████▍ | 22/26 [00:00<00:00, 15.55it/s]


100%|██████████| 26/26 [00:00<00:00, 15.32it/s]




  3%|▎         | 3/98 [00:01<00:35,  2.70it/s]


  0%|          | 0/26 [00:00<?, ?it/s]


 85%|████████▍ | 22/26 [00:00<00:00, 204.82it/s]


100%|██████████| 26/26 [00:00<00:00, 203.29it/s]

  4%|▍         | 4/98 [00:01<00:28,  3.35it/s]


  0%|          | 0/26 [00:00<?, ?it/s]


 15

 85%|████████▍ | 22/26 [00:05<00:01,  2.45it/s]


 88%|████████▊ | 23/26 [00:05<00:01,  2.41it/s]


 96%|█████████▌| 25/26 [00:07<00:00,  2.15it/s]


100%|██████████| 26/26 [00:08<00:00,  1.10it/s]




 46%|████▌     | 45/98 [10:17<06:01,  6.82s/it]


  0%|          | 0/26 [00:00<?, ?it/s]


  4%|▍         | 1/26 [02:00<50:21, 120.87s/it]


 23%|██▎       | 6/26 [02:13<28:26, 85.34s/it] 


 27%|██▋       | 7/26 [02:21<19:44, 62.33s/it]


 31%|███       | 8/26 [02:34<14:16, 47.58s/it]


 42%|████▏     | 11/26 [02:43<08:32, 34.15s/it]


 62%|██████▏   | 16/26 [03:36<04:31, 27.10s/it]


 85%|████████▍ | 22/26 [05:40<01:40, 25.15s/it]


 88%|████████▊ | 23/26 [10:30<05:13, 104.61s/it]


 96%|█████████▌| 25/26 [11:41<01:23, 83.99s/it] 


100%|██████████| 26/26 [14:14<00:00, 104.46s/it]




 47%|████▋     | 46/98 [24:31<3:46:12, 261.00s/it]


  0%|          | 0/26 [00:00<?, ?it/s]


  4%|▍         | 1/26 [00:00<00:06,  3.90it/s]


 31%|███       | 8/26 [00:00<00:03,  5.38it/s]


 62%|██████▏

 31%|███       | 8/26 [00:00<00:00, 76.84it/s]


 85%|████████▍ | 22/26 [00:00<00:00, 80.24it/s]


100%|██████████| 26/26 [00:00<00:00, 71.30it/s]

 87%|████████▋ | 85/98 [27:50<01:05,  5.04s/it]


  0%|          | 0/26 [00:00<?, ?it/s]


  4%|▍         | 1/26 [00:00<00:02,  9.20it/s]


 38%|███▊      | 10/26 [00:00<00:01, 12.56it/s]


 65%|██████▌   | 17/26 [00:00<00:00, 16.66it/s]


 85%|████████▍ | 22/26 [00:00<00:00, 19.32it/s]


100%|██████████| 26/26 [00:00<00:00, 19.08it/s]




 88%|████████▊ | 86/98 [27:51<00:44,  3.74s/it]


  0%|          | 0/26 [00:00<?, ?it/s]


 54%|█████▍    | 14/26 [00:00<00:00, 116.05it/s]


 85%|████████▍ | 22/26 [00:00<00:00, 98.36it/s] 


100%|██████████| 26/26 [00:00<00:00, 78.35it/s]

 89%|████████▉ | 87/98 [27:51<00:29,  2.72s/it]


  0%|          | 0/26 [00:00<?, ?it/s]


 81%|████████  | 21/26 [00:00<00:00, 200.66it/s]


100%|██████████| 26/26 [00:00<00:00, 179.11it/s]

 90%|████████▉ | 88/98 [27:51<00:19,  1.95s/it]


  0%|          | 0/26 [00:

In [37]:
with open('perf_analysis.json', 'w') as fileobj:
    def transform_result(result):
        return {
            'sexpr_set': result['sexpr_set'],
            'logic': result['logic'],
            'check_time': result['check_time'],
            'check_result': result['check_result'].__repr__()
        }
    fileobj.write(json.dumps([[transform_result(y) for y in x] for x in test_results]))

[(x['check_result'], x['check_time'], x['logic']) for x in test_results[10]]

[(unsat, 0.072483, None),
 (sat, 0.001604, 'AUFLIA'),
 (sat, 0.001463, 'AUFLIRA'),
 (sat, 0.001307, 'AUFNIRA'),
 (sat, 0.002029, 'LRA'),
 (unsat, 0.014678, 'QF_ABV'),
 (unsat, 0.016055, 'QF_AUFBV'),
 (unsat, 0.031579, 'QF_UFBV'),
 (sat, 0.001559, 'QF_AUFLIA'),
 (sat, 0.005869, 'QF_AX'),
 (unsat, 0.015033, 'QF_BV'),
 (sat, 0.001708, 'QF_IDL'),
 (unknown, 0.003935, 'QF_RDL'),
 (sat, 0.002036, 'QF_LIA'),
 (sat, 0.002314, 'QF_LRA'),
 (unsat, 0.042617, 'QF_NIA'),
 (unknown, 0.0021, 'QF_NRA'),
 (unknown, 0.000861, 'QF_UF'),
 (sat, 0.006092, 'QF_UFIDL'),
 (sat, 0.00722, 'QF_UFLIA'),
 (sat, 0.007153, 'QF_UFLRA'),
 (unsat, 0.074062, 'QF_UFNRA'),
 (unsat, 0.019904, 'UFLRA'),
 (sat, 0.001701, 'UFNIA'),
 (unsat, 0.062735, 'UFBV'),
 (unsat, 0.065038, 'QF_S')]

In [6]:
with open('./perf_analysis.json', 'r') as fileobj:
    test_results = json.load(fileobj)

In [18]:
correct_answer = {x[0]['sexpr_set']: x[0]['check_result'] for x in test_results} # Get the value for the default

from collections import Counter

number_correct = Counter()
total_time = Counter()

for sat_set in test_results:
    for answer in sat_set:
        logic = answer['logic']
        total_time[logic] += answer['check_time']
        if correct_answer[answer['sexpr_set']] == answer['check_result']:
            number_correct[logic] += 1

number_correct

valid_logics = [k for k, v in number_correct.items() if v == number_correct[None]]
valid_logic_times = {k:v for k,v in total_time.items() if k in valid_logics}
valid_logic_times

{None: 277.02209099999993,
 'QF_ABV': 24.912236,
 'QF_AUFBV': 21.434394,
 'QF_UFBV': 26.776957999999993,
 'QF_BV': 20.227349999999976,
 'QF_NIA': 113.02815199999998,
 'QF_UFNRA': 293.1113659999998,
 'UFLRA': 355.91857999999985,
 'UFBV': 227.296832,
 'QF_S': 329.6392020000001}

In [20]:
best_time = min([v for _, v in valid_logic_times.items()])
print({k:v for k,v in valid_logic_times.items() if v == best_time})

{'QF_BV': 20.227349999999976}
